# 実践その参（言語処理パート）

HugginefaceのRobertモデルを使って、ジャンルを分類する。

やり方の細かい部分は問いません。動けばOKです。

想定される実際のタスク:
「コールセンターのやりとりを問い合わせとクレームの2種類に分類する」など。


## 条件
- HugginefaceのRobertaモデルを使うこと。
- 学習のループは自分で書くこと
- テスト用のパラグラフが正しく推論できること

In [ ]:
# ここにStudent IDを記載しておいてください。
Student_ID = ""

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import

import os
import numpy as np
import math
import random
import torch

import random
random.seed(55)

# データの準備


In [ ]:
# livedoorニュースのデータセットをダウンロードして、解凍します。

! wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
! tar xvzf ldcc-20140209.tar.gz

In [ ]:
# カテゴリーの一覧は以下の通りです。
category_list = [
    "dokujo-tsushin",
    "it-life-hack",
    "kaden-channel",
    "livedoor-homme",
    "movie-enter",
    "peachy",
    "smax",
    "sports-watch",
    "topic-news",
]

In [ ]:
# ディレクトリをチェックして、リストに格納します。

import glob

# read files
files1 = glob.glob('./text/*/*.txt')

files2 = []
for item in files1:
    if "LICENSE.txt" not in item:
        files2.append(item)

len(files2)

# Haggingface

本プロジェクトではHaggingfaceのソースコードを利用します。

https://github.com/huggingface/transformers


In [ ]:
# transformersをインストールします。
!pip install transformers --upgrade

In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, RobertaForMaskedLM, RobertaForSequenceClassification

# トークナイザー

In [ ]:
# tokenizer
# 本当はトークナイザーも自作できた方が良いのですが、作成にはそこそこパワーのあるマシンが必要になるため、使えるものをご提供します。

# トークナイザをダウンロード
! wget https://payloadcms.shabelab.com/assets/datasets/roberta_base_tokenizer_0511.tar.gz

# ダウンロードしたトークナイザを解凍
! tar xvzf roberta_base_tokenizer_0511.tar.gz

In [ ]:
# トークナイザをインスタンス化
tokenizer = AutoTokenizer.from_pretrained("./roberta_base_tokenizer_0511")

In [ ]:
# トークナイザをテスト
tokenizer("今日の晩御飯はカレーでした。")

In [ ]:
tokenizer.decode([0, 8296, 27403, 1751, 2908, 289, 14114, 4215, 283, 2])

# Datasetの準備

ヒント動画を参考にしても良いですし、他のリソースを参考にしていただいても構いません。

バッチデータを以下のような感じで取得して情報を出力すると
```
one_batch = next(iter(train_loader))
print("one_batch count: ", len(one_batch))
print("")
print("one_batch.0.shape: ", one_batch[0].shape)
print("one_batch.1.shape: ", one_batch[1].shape)
print("one_batch.2.shape: ", one_batch[2].shape)
print("")
print("one_batch.1: ", one_batch[1])
print("one_batch.2: ", one_batch[2])
```

以下のようなデータが出力されます。
```
one_batch count:  3

one_batch.0.shape:  torch.Size([2, 512])
one_batch.1.shape:  torch.Size([2, 512])
one_batch.2.shape:  torch.Size([2, 1])

one_batch.1:  tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
one_batch.2:  tensor([[0],
        [8]])
```

取得できるデータは順番に実際のインプットデータ、マスクデータ、そして正解ラベルとなっている必要があります。
マスクデータはblock_sizeが512でインプットするデータのトークン数が512以上の場合（512までで残りはカットされます）は512個の1がならび、512未満の場合はトークンの存在するところまでは1でそれ以外（パディングで埋められた部分）が0となります。

ところで、一つのバッチに入ってきたデータのトークン数が150と170など、いずれも最大直に満たない場合は、大きな方の値（この例では170）に揃えるようにしてください。

ヒント1: 講師はDataset型を継承したMyDatasetを定義し、バッチ化する際にcollateファンクションを使ってマスクを作成しています。  
ヒント2: transformersを使ったモデルが重たいので、バッチサイズは2くらいで十分です。

# アーキテクチャ

In [ ]:
# スクラッチでゼロから学習を行うには流石にデータセットが少ないので、pre-trainedのウェイトをダウンロードし、
# そこからファインチューニングを書けることにします。
! wget https://payloadcms.shabelab.com/assets/datasets/roberta-pretrain-0-1500000.tar.gz

In [ ]:
# ダウンロードしてきたウェイトを解凍します。
! tar xvaf roberta-pretrain-0-1500000.tar.gz

In [ ]:
# change number of lables

# mm_config.num_labels = 9
model = RobertaForSequenceClassification.from_pretrained(
    "./roberta-pretrain-0-1500000/", num_labels=9)

model.classifier


# トレーニングループ

下記を参考に学習ループを自分で作成しましょう。  
https://huggingface.co/transformers/model_doc/roberta.html#robertaforsequenceclassification

ヒント: GoogleのColabは1時間触らないでいるとタイムアウトします。学習には最低でも数時間かかります。  
途中で止めて、そこまでのウェイトを保存、ダウンロードして、次回をそれをアップロード、読み込むことで途中から始めることも可能です。

In [ ]:
# optimizer
# 下記はadamを使っていますが、他のものを使ってもokです。

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [ ]:
# device

device = torch.device("cuda:0")
model = model.to(device)


In [ ]:
# 学習ループ
# ここに for ループを作成してください。


In [ ]:
# save
model.save_pretrained("checkpoint-1")

# テスト

下記は変更の必要はありません。  
"sports-watch"として推論できれば正解です。


In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, RobertaForMaskedLM

In [ ]:
model2 = model.from_pretrained("checkpoint-1")
model2.classifier

In [ ]:
sample = "前田智徳氏、走攻守そろった新人・近本光司を称賛　「ルーキーではなかなかできません」0LINE共有ボタン2019年5月23日 13時30分 Sports Watch22日放送、テレビ朝日「報道ステーション」に、野球解説者の前田智徳氏が出演。阪神タイガースの新人・近本光司を称賛した。同日の東京ヤクルトスワローズ戦で、近本は守備でチームを救った。1-1の同点で迎えた5回表、二死二塁のピンチで、見事なバックホームから失点を防いだ。危機を脱した阪神は、7回ウラに糸井嘉男のタイムリーで勝ち越し、連勝を飾っている。前田氏は近本の守備を称賛したうえで、「注目は足」と述べた。この日の近本は、1回に内野安打で出塁すると盗塁成功。5回にも三盗を決めている。盗塁数はリーグトップの13。新人で盗塁王となれば、39盗塁を記録した2001年の赤星憲広以来、18年ぶりの快挙だ。前田氏は「十分にチャンスはある」と、近本のさらなる盗塁に太鼓判を押す。さらに、前田氏は「軸足にしっかり体重を乗せて、レフト（逆方向）へ力強い打球を打てる」と、近本の打撃にも称賛。「ルーキーではなかなかできません」と、走攻守の三拍子がそろった近本が新人離れしていると賛辞を寄せた。"

input_ids = tokenizer(
        sample,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=512,
        truncation=True,).input_ids

input_ids.shape


In [ ]:
resuslts = model2(input_ids=input_ids)

logits = resuslts.logits
logits.shape

In [ ]:
_index = logits.argmax().item()
_index

In [ ]:
category_list[_index]